In [1]:
import requests
from bs4 import BeautifulSoup as bs
import time
import os

In [2]:
#爬虫
#获取前十页帖子的链接
all_note_link=set()
for i in range(10):
    res=requests.get("https://tieba.baidu.com/f?kw=%E5%9B%BD%E9%99%85%E7%B1%B3%E5%85%B0&ie=utf-8&pn="+str(i*50))
    soup=bs(res.text,'lxml')
    all_note_link=all_note_link|{div.a['href'] for div in soup.find_all('div',class_="threadlist_title")}
print('链接数：',len(all_note_link))

链接数： 498


In [ ]:
#爬取所有帖子的内容（楼中楼无法爬取）
today='_'.join([str(t) for t in time.localtime( time.time())[:3]])
path='text/'+today
if not os.path.isdir(path):
    os.mkdir(path)

for i,link in enumerate(sorted(list(all_note_link))):
    print(i,len(all_note_link))
    res=requests.get("http://tieba.baidu.com"+link)
    soup=bs(res.text,'lxml')
    text_list=soup.find_all('div',class_="j_d_post_content")
    with open(path+'/'+link[3:]+'.txt','w',encoding='utf8') as f:
        try:
            f.write(soup.h3.text+'\n')
        except Exception as e:
            print(e)
        for t in text_list:
            f.write(t.text.strip()+'\n')

In [87]:
#词云
import jieba
import jieba.posseg as pseg
from pyecharts import WordCloud

jieba.add_word('不能娶')
jieba.add_word('舔狗')

all_file=os.listdir(path)
all_word=[]
#停词表
with open('text/stop.txt') as f:
    stopwords=set(f.read().split())|{' ','\n','\u3000'}
#分词
for file in all_file:
    with open(path+'/'+file,encoding='utf8') as f:
        for w,c in pseg.cut(f.read()):
            if len(w)>1 and (c[0]=='n' or c=='a' or c=='x') and w not in stopwords:
                all_word.append(w)
#统计所有词             
word_count={}

for w in all_word:
    if w in word_count:
        word_count[w]+=1
    else:
        word_count[w]=1

In [88]:
word_count_list=[[k,word_count[k]] for k in word_count]
word_count_list=sorted(word_count_list,key=lambda k:k[1],reverse=True)

In [89]:
wordcloud = WordCloud(width=900, height=500)
wordcloud.add("",[t[0] for t in word_count_list[:200]], [t[1] for t in word_count_list[:200]],
              shape= 'triangle')
wordcloud